# Shamir Secret Sharing

First, let's import some "helper" functions for dealing with polynomials.

In [ ]:
from helper import *
import matplotlib.pyplot as plt

## Share
Here is our sharing function. It takes as input `s` (the secret), `n` (the number of parties to share amongst), `t` (the degree of the polynomial used for sharing, so t+1 is the minimum number of parties needed to reconstruct the secret), and `p` (**REMOVE THIS?**). The input `printout` tells the function whether it should print an explanation while it's working or not.

In [ ]:
def share(s, n, t, p, printout=False):
    # check p
    if not isprime(p):
        print("p={} is not prime!".format(p))
        return
    if p <= n:
        print("p={} must be greater than n={}".format(p, n))
        return
        
    # check t
    if t >= n:
        print("t={} must be less than n={}".format(t, n))
        return

    # check s is in field
    if p <= s:
        print("s={} must be less than p={}".format(s, p))
        return
        
    ### the interesting part ###########################
    coeffs = []
    for i in range(t):
        # sample coefficients from F_p = {0, ..., p-1}
        coeffs.append(randint(0,p-1))
    # secret is the y-intercept / constant term of the poly
    coeffs.append(s)
    
    shares = []
    for i in range(1,n+1):
        shares.append((i, eval_poly(coeffs, i)))
    #####################################################
        
    if printout:
        # plot the polynomial
        print("The random degree t={} polynomial is".format(t))
        print_poly(coeffs)
        x = linspace(0, n, n+1)
        y = [eval_poly(coeffs, i) for i in x]
        plt.plot(x, y)
    
        # plot the shares
        print()
        print("The shares are points on that polynomial:")
        print(shares)
        x1 = [shares[i][0] for i in range(len(shares))]
        y1 = [shares[i][1] for i in range(len(shares))]
        plt.scatter(x1, y1)
    
        # plot the secret
        plt.scatter(0, s, color='orange')
    
        print()
        print("Here is a visual representation (secret in orange).")
        
    return shares

For example, say our secret is the number 42. We'd like to share it among 10 parties (n=10), and we'll allow any 4 of those to recover the secret (t+1 can recover, so t=3). Now let's pick a prime number p that's bigger than both the secret and the number of parties (so, p>42). The next largest prime is 43, so let's try that!

How would you call the `share` function with these parameters?

**Answer**
```
shares = share(42, 10, 3, 43)
```

In [ ]:
shares = share(42, n=10, t=3, p=43, printout=True)

Now we can distribute these points among our 10 parties!

## Reconstruct
Say 4 of those parties (Alice, Bob, Charlie, and Diane) want to recover the secret. We set $t=3$, so they should be able to do this (remember, a minimum of $t+1$ parties is needed). Together, they hold 4 points on the degree-3 polynomial, which uniquely defines it! They can pool this information to recover the polynomial $f$ and evaluate it at $x=0$ using the `recon` function:

In [ ]:
def recon(shares, n, t):
    if len(shares) < t+1:
        print("Not enough shares to reconstruct! ({} < t+1={})".format(len(shares), t+1))
        return
    
    # plot submitted shares on poly plot?
    
    # i Lagrange basis polynomials evaluated at 0
    ell = [1]*len(shares)
    for i in range(len(shares)):
        for j in range(len(shares)):
            if i!=j:
                ell[i] *= float(0-shares[j][0])/(shares[i][0]-shares[j][0])
    
    # interpolate
    # f(X) = sum_1^{t+1} ell_i(X) * y_i
    # s = f(0)
    s = 0
    for i in range(len(shares)):
        s += ell[i]*shares[i][1]

    print("The reconstructed secret is:")
    return int(s)

Assuming Alice, Bob, Charlie, and Diane have the points for $x=1,2,3,4$, respectively, can you use the shares of 42 we created earlier to recover the secret?

**Answer**
```
recon(shares[:4], n=10, t=3)
```

In [ ]:
recon_s = recon([shares[0], shares[1], shares[4], shares[5]], n=10, t=3, printout=True)

## Sharing messages
So far we've shared only numbers. This might seem limiting, but the whole world can be represented in numbers! For instance, we can share messages by turning letters into numbers:

| a | b | c | d | e | f | g | h | i | j | k | l | m | n | o | p | q | r | s | t | u | v | w | x | y | z |
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20 | 21 | 22 | 23 | 24 | 25 | 26 |

Then to send the message "Hello world!", we turn it into a list of numbers: `[8, 5, 12, 12, 15, 23, 15, 18, 12, 4]` (for now, to keep things simple, we'll ignore spaces, punctuation, and any other symbols). Then we can share each of the numbers in the list, and each party will have a list of numbers that are shares of the message.

The functions `share_msg` and `recon_msg` do exactly this. `share_msg` converts each letter in a message to a number and shares each of those numbers. It outputs a list of shares (points) for each party. Notice that the x-coordinate of the points is the same for all the points in one party's set! `recon_msg` takes each party's list of points and reconstructs each letter; then it outputs the recovered message (since we didn't consider spaces, punctuation, etc., those are missing from the reconstructed message).

In [ ]:
def share_msg(msg, n, t, p=29):
    arr = convert_msg(msg)
    shares_by_letter = []
    for x in arr:
        shares_by_letter.append(share(x, n, t, p))
    
    shares_by_party = []
    for i in range(n):
        share_i = [row[i] for row in shares_by_letter]
        shares_by_party.append(share_i)
        print("Party {}'s share is: {}".format(i+1, share_i))
    return shares_by_party

def recon_msg(arr_shares, n, t):
    ans = []
    for i in range(len(arr_shares[0])):
        letter_shares = [arr[i] for arr in arr_shares]
        ans.append(recon(letter_shares, n, t))
    return nums_to_msg(ans)

Here is how we share our example message "Hello world!" among 5 parties so that any t+1=3 can recover:

In [ ]:
msg_shares = share_msg("Hello world!", n=5, t=2)

And here it is, reconstructed using all the shares:

In [ ]:
recon_msg(msg_shares, n=5, t=5)

We can also reconstruct it using only 3 (or 4) shares:

In [ ]:
recon_msg([msg_shares[0], msg_shares[1], msg_shares[3]], n=5, t=2)

## Now it's your turn!